# Build a Medium Posting Search Engine using Pinecone

#Load Data

In [1]:
!pip3 install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!pip3 install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 11.2 MB/s eta 0:00:00


# Download Medium article data

In [3]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/medium_data.csv

--2025-04-16 19:21:36--  https://s3-geospatial.s3.us-west-2.amazonaws.com/medium_data.csv
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 3.5.80.192, 52.218.154.41, 52.218.176.137, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|3.5.80.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 599580 (586K) [text/csv]
Saving to: ‘medium_data.csv’

medium_data.csv     100%[===================>] 585.53K   799KB/s    in 0.7s    

2025-04-16 19:21:38 (799 KB/s) - ‘medium_data.csv’ saved [599580/599580]



In [4]:
!wc -l medium_data.csv

2499 medium_data.csv


# Preprocess Medium data and save it as a file

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("medium_data.csv")

In [7]:
df.head()

,id,url,title,subtitle,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/not-all-rainbow...,Not All Rainbows and Sunshine: The Darker Side...,Part 1: The Risks and Ethical Issues…,453.0,11,9,Towards Data Science,27-01-2023
1,2,https://towardsdatascience.com/ethics-in-ai-po...,Ethics in AI: Potential Root Causes for Biased...,An alternative approach to understanding bias ...,311.0,3,12,Towards Data Science,27-01-2023
2,3,https://towardsdatascience.com/python-tuple-th...,"Python Tuple, The Whole Truth and Only the Tru...",NaN,188.0,0,24,Towards Data Science,27-01-2023
3,4,https://towardsdatascience.com/dates-and-subqu...,Dates and Subqueries in SQL,Working with dates in SQL,15.0,1,4,Towards Data Science,27-01-2023
4,5,https://towardsdatascience.com/temporal-differ...,Temporal Differences with Python: First Sample...,NaN,10.0,0,13,Towards Data Science,27-01-2023


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2498 entries, 0 to 2497
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            2498 non-null   int64  
 1   url           2498 non-null   object 
 2   title         2498 non-null   object 
 3   subtitle      2073 non-null   object 
 4   claps         2423 non-null   float64
 5   responses     2498 non-null   int64  
 6   reading_time  2498 non-null   int64  
 7   publication   2498 non-null   object 
 8   date          2498 non-null   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 175.8+ KB


# Data Cleanup

In [10]:
df['title'] = df['title'].astype(str).fillna('')
df['subtitle'] = df['subtitle'].astype(str).fillna('')

In [11]:
df['metadata'] = df.apply(lambda row: {'title': row['title'] + " " + row['subtitle']}, axis=1)

In [12]:
df.head()

,id,url,title,subtitle,claps,responses,reading_time,publication,date,metadata
0,1,https://towardsdatascience.com/not-all-rainbow...,Not All Rainbows and Sunshine: The Darker Side...,Part 1: The Risks and Ethical Issues…,453.0,11,9,Towards Data Science,27-01-2023,{'title': 'Not All Rainbows and Sunshine: The ...
1,2,https://towardsdatascience.com/ethics-in-ai-po...,Ethics in AI: Potential Root Causes for Biased...,An alternative approach to understanding bias ...,311.0,3,12,Towards Data Science,27-01-2023,{'title': 'Ethics in AI: Potential Root Causes...
2,3,https://towardsdatascience.com/python-tuple-th...,"Python Tuple, The Whole Truth and Only the Tru...",nan,188.0,0,24,Towards Data Science,27-01-2023,"{'title': 'Python Tuple, The Whole Truth and O..."
3,4,https://towardsdatascience.com/dates-and-subqu...,Dates and Subqueries in SQL,Working with dates in SQL,15.0,1,4,Towards Data Science,27-01-2023,{'title': 'Dates and Subqueries in SQL Working...
4,5,https://towardsdatascience.com/temporal-differ...,Temporal Differences with Python: First Sample...,nan,10.0,0,13,Towards Data Science,27-01-2023,{'title': 'Temporal Differences with Python: F...


# Prep for Upsert

In [24]:
from google.colab import userdata


API_KEY = userdata.get('pinecone_key')

In [25]:
from pinecone import Pinecone

pc = Pinecone(api_key=API_KEY)

In [26]:
from pinecone import ServerlessSpec


spec = ServerlessSpec(
    cloud="aws",
    region="us-east-1"
)

In [27]:
index_name = 'semantic-search-fast'

In [28]:
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

In [29]:
existing_indexes

['semantic-search-fast']

In [30]:
import time


try:
    pc.delete_index(index_name)
except Exception as e:
    print(e)

pc.create_index(
    index_name,
    dimension=384,
    metric='dotproduct',
    spec=spec
)

while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [31]:
from sentence_transformers import SentenceTransformer
import torch

# Generate sentence embeddings and ingest them into Pinecone

In [32]:
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [33]:

df['values'] = df['metadata'].map(
    lambda x: (model.encode(x["title"])).tolist())

In [34]:
df.head()

,id,url,title,subtitle,claps,responses,reading_time,publication,date,metadata,values
0,1,https://towardsdatascience.com/not-all-rainbow...,Not All Rainbows and Sunshine: The Darker Side...,Part 1: The Risks and Ethical Issues…,453.0,11,9,Towards Data Science,27-01-2023,{'title': 'Not All Rainbows and Sunshine: The ...,"[0.0040841964073479176, 0.038495469838380814, ..."
1,2,https://towardsdatascience.com/ethics-in-ai-po...,Ethics in AI: Potential Root Causes for Biased...,An alternative approach to understanding bias ...,311.0,3,12,Towards Data Science,27-01-2023,{'title': 'Ethics in AI: Potential Root Causes...,"[-0.0701502338051796, 0.024755671620368958, -0..."
2,3,https://towardsdatascience.com/python-tuple-th...,"Python Tuple, The Whole Truth and Only the Tru...",nan,188.0,0,24,Towards Data Science,27-01-2023,"{'title': 'Python Tuple, The Whole Truth and O...","[-0.0594332329928875, -0.027460753917694092, 0..."
3,4,https://towardsdatascience.com/dates-and-subqu...,Dates and Subqueries in SQL,Working with dates in SQL,15.0,1,4,Towards Data Science,27-01-2023,{'title': 'Dates and Subqueries in SQL Working...,"[0.002338135614991188, -0.0032879936043173075,..."
4,5,https://towardsdatascience.com/temporal-differ...,Temporal Differences with Python: First Sample...,nan,10.0,0,13,Towards Data Science,27-01-2023,{'title': 'Temporal Differences with Python: F...,"[-0.08383707702159882, -0.029564589262008667, ..."


In [35]:
df['id'] = df.reset_index(drop = 'index').index

In [36]:

df_upsert = df[['id', 'values', 'metadata']]

In [37]:
df_upsert['id'] = df_upsert['id'].map(lambda x: str(x))

<ipython-input-37-5f02da224420>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upsert['id'] = df_upsert['id'].map(lambda x: str(x))


In [38]:
df_upsert.head()

,id,values,metadata
0,0,"[0.0040841964073479176, 0.038495469838380814, ...",{'title': 'Not All Rainbows and Sunshine: The ...
1,1,"[-0.0701502338051796, 0.024755671620368958, -0...",{'title': 'Ethics in AI: Potential Root Causes...
2,2,"[-0.0594332329928875, -0.027460753917694092, 0...","{'title': 'Python Tuple, The Whole Truth and O..."
3,3,"[0.002338135614991188, -0.0032879936043173075,...",{'title': 'Dates and Subqueries in SQL Working...
4,4,"[-0.08383707702159882, -0.029564589262008667, ...",{'title': 'Temporal Differences with Python: F...


In [39]:
index = pc.Index(index_name)

In [40]:
index.upsert_from_dataframe(df_upsert) # 6k takes 1 min

sending upsert requests:   0%|          | 0/2498 [00:00<?, ?it/s]

{'upserted_count': 2498}

# Query

In [41]:
xc = index.query(vector=(model.encode("what is ethics in AI")).tolist(), # python list
           top_k=10,
           include_metadata=True,
           include_values=True)

In [43]:
for result in xc['matches']:
    print(result['id'], result['score'], result['metadata'])

1 0.74098587 {'title': 'Ethics in AI: Potential Root Causes for Biased Algorithms An alternative approach to understanding bias in\xa0data'}
1634 0.74098587 {'title': 'Ethics in AI: Potential Root Causes for Biased Algorithms An alternative approach to understanding bias in\xa0data'}
1326 0.717011333 {'title': 'The ethical implications of AI in\xa0design It’s time to talk ethics, chatGPT, Midjourney &\xa0more.'}
661 0.666943789 {'title': 'Ethical Considerations In Machine Learning\xa0Projects Don’t forget these topics when building AI\xa0systems'}
1627 0.651700854 {'title': 'Navigating the Ethical Contours of AI Copy Generators Like\xa0ChatGPT Be prepared to cite the\xa0AI…'}
2029 0.651700854 {'title': 'Navigating the Ethical Contours of AI Copy Generators Like\xa0ChatGPT Be prepared to cite the\xa0AI…'}
1344 0.606802464 {'title': 'Using AI in UX research: a structured and ethical\xa0approach A structured approach for assessing the…'}
1830 0.596021116 {'title': 'The Path towards AI Reg